In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
!pip install rouge-score
!pip install nltk

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=71efad1caf00304efae227792a360e1e2e2184e7d02ed58878535023de7ef335
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import json
from datasets import load_metric
from transformers import pipeline


# model_checkpoint = "mylas02/BERT_SQuaD_FineTuned"
# model_checkpoint = "mylas02/XLNET_SQuaD_FineTuned_v2"
model_checkpoint = "mylas02/Roberta_SQuaD_FineTuned"
question_answerer = pipeline("question-answering", model= model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [7]:
import json
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from collections import Counter

def rouge_n(predicted, actual, n):
    predicted_tokens = word_tokenize(predicted.lower())
    actual_tokens = word_tokenize(actual.lower())

    predicted_ngrams = list(ngrams(predicted_tokens, n))
    actual_ngrams = list(ngrams(actual_tokens, n))

    overlap_count = sum((Counter(predicted_ngrams) & Counter(actual_ngrams)).values())
    predicted_count = len(predicted_ngrams)
    actual_count = len(actual_ngrams)

    precision = overlap_count / predicted_count if predicted_count > 0 else 0
    recall = overlap_count / actual_count if actual_count > 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_score

def rouge(predicted, actual, n):
    precision, recall, f1_score = rouge_n(predicted, actual, n)
    return precision, recall, f1_score


with open("PFW_dataset_final.json", "r") as file:
    data = json.load(file)

exact_match_count = 0
total_questions = 0
f1_scores = []
bleu_scores = []
rouge_precision_total = 0
rouge_recall_total = 0
rouge_f1_score_total = 0


for example in data["data"]:
    for paragraph in example["paragraphs"]:
        context = paragraph["context"]
        for qa in paragraph["qas"]:
            question = qa["question"]
            predicted_answer = question_answerer(question=question, context=context)["answer"]
            actual_answer = qa["answers"]


            f1_scores.append(question_answerer(question=question, context=context)["score"])


            actual_tokens = actual_answer[0]["text"].split()
            bleu_scores.append(sentence_bleu([predicted_answer.split()], actual_tokens))


            rouge_precision, rouge_recall, rouge_f1_score = rouge(predicted_answer, actual_answer[0]["text"], n=2)
            rouge_precision_total += rouge_precision
            rouge_recall_total += rouge_recall
            rouge_f1_score_total += rouge_f1_score


            exact_match_count += any(predicted_answer == answer["text"] for answer in actual_answer)
            total_questions += 1


avg_f1_score = sum(f1_scores) / len(f1_scores)
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
avg_rouge_precision = rouge_precision_total / total_questions
avg_rouge_recall = rouge_recall_total / total_questions
avg_rouge_f1_score = rouge_f1_score_total / total_questions
exact_match_percentage = (exact_match_count / total_questions) * 100

print("Average F1 Score:", avg_f1_score)
print("Average BLEU Score:", avg_bleu_score)
print("Average ROUGE Precision:", avg_rouge_precision)
print("Average ROUGE Recall:", avg_rouge_recall)
print("Average ROUGE F1 Score:", avg_rouge_f1_score)
print("Exact Match Percentage:", exact_match_percentage)


Average F1 Score: 0.49534285135439315
Average BLEU Score: 0.33690953063672824
Average ROUGE Precision: 0.6682896772955831
Average ROUGE Recall: 0.49831392423136817
Average ROUGE F1 Score: 0.5264614853697066
Exact Match Percentage: 31.10236220472441
